In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns
import sklearn.metrics as metrics
import umap

from fast_pareto import nondominated_rank
from joblib import delayed, Parallel
from os import getcwd
from scipy.spatial.distance import cdist
from scipy.stats import entropy
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import mutual_info_classif
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Set random seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Set plot formatting parameters
sns.set_context('paper')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] 

# Set multiprocessing jobs (-1 to use all processors)
N_JOBS = -1

# Set input/output file directory
DIRPATH = getcwd() + r'\{}'

# Load single-concentration data
sc_df = pd.read_csv(DIRPATH.format('single_conc.csv')).iloc[:, np.r_[0, 4:8]].rename(columns={
    'Library': 'LIB', 'Max conc': 'CONC_MAX', 'Median % activity': 'PACT_MED', 'Tested in CR?': 'CR_TESTED'
}).dropna().set_index('DTXSID')

# Load QAQC annotation data
qaqc_df = pd.read_csv(DIRPATH.format('TTR-qaqc-annotations.csv')).set_index('DTXSID')
# Join QAQC data onto single-concentration data and select passed compounds only
sc_df = sc_df.join(qaqc_df['QualityCall'], how='inner')
sc_pass_df = sc_df[sc_df['QualityCall'] == 'Pass'].drop(columns=['QualityCall'])

# Load assay key table
key_df = pd.read_csv(DIRPATH.format('AssayKey.csv')).set_index('EPA_SAMPLE_ID')
# Load raw assay data and select only non-control, single-concentration assay samples
sp_query = 'spid.str.startswith(\'EPAPLT\') and assay_type == \'hTTR_SingleConc_ToxCast\''
sp_df = pd.read_csv(DIRPATH.format('hTTR_dat_all.csv')).query(sp_query)
# Compile samplewise activity statistics and chemical IDs from raw data and key table
sp_agg_df = pd.DataFrame(
    sp_df.groupby('spid').agg(
        PACT_MED=('pactivity', 'median'),
        PACT_SD=('pactivity', 'std'),
        PACT_MIN=('pactivity', 'min'),
        PACT_MAX=('pactivity', 'max'),
        PACT_RNG=('pactivity', lambda x: x.max() - x.min())
    )
).join(key_df['DTXSID'], how='inner')

# Define activity bins based on median SD of triplicate measurements from raw assay data
bin_width = 4 * sp_agg_df['PACT_SD'].median()
n_bins = int(np.round(100 / bin_width))
bin_edges = bin_width * range(0, n_bins + 1)

# Remove very negative values
pact_neg_thresh = -15
sc_pass_df = sc_pass_df[sc_pass_df['PACT_MED'] > pact_neg_thresh]
# Discretize single-concentration data into activity classes
sc_pass_df['PACT_CLASS'] = np.digitize(sc_pass_df['PACT_MED'], bin_edges)

# Output summary table of min and max activity values and counts by class
cls_agg_df = sc_pass_df.groupby('PACT_CLASS').agg({'PACT_MED': ['min', 'max', 'median', 'size']})
cls_agg_df['MIDPOINT'] = [None] + list((bin_edges[1:] + bin_edges[:-1]) / 2) + [None]
cls_agg_df['DELTA'] = cls_agg_df['MIDPOINT'] - cls_agg_df['PACT_MED']['median']
cls_agg_df.head(n_bins + 2)

In [ ]:
# Plot histogram and KDE of activity data using predefined bin edges
fig, ax = plt.subplots()
plt_bin_edges = [pact_neg_thresh] + list(bin_edges) + [max(sc_pass_df['PACT_MED'])]
sns.histplot(data=sc_pass_df, x='PACT_MED', bins=plt_bin_edges, ax=ax, kde=True)
for i in range(len(plt_bin_edges) - 1):
    plt.text((plt_bin_edges[i] + plt_bin_edges[i + 1]) / 2, 10, str(int(i)), ha='center', va='bottom')
ax.set_xlabel('Median % Activity')
ax.set_title('Activity Distribution in Final Data')
plt.show()

In [ ]:
# Load Padel descriptors and remove duplicates
pd_df = pd.read_csv(DIRPATH.format('qsar_ready_PadelDesc.csv')).set_index('Name').drop_duplicates(keep=False)
# Join Padel descriptors onto single-concentration data
sc_pass_df = sc_pass_df.join(pd_df, how='inner')

# Embedding selected manually on chemical principles
embedding = ['CrippenLogP', 'nHBAcc', 'nHBDon', 'naAromAtom', 'C1SP3', 'C1SP2', 'VE1_DzZ', 'maxdssC', 'hmin', 'hmax'] \
    + ['ZMIC{}'.format(i) for i in range(1, 6)] \
    + ['ATSC{}c'.format(i) for i in range(0, 6)] \
    + ['ATSC{}m'.format(i) for i in range(0, 6)] \
    + ['ETA_dAlpha_A', 'ETA_Shape_Y', 'ETA_BetaP_s', 'ETA_Beta_ns_d']

# Assemble X and y matrices for model training from final single-concentration data and selected embedding
X = sc_pass_df[embedding]
y = sc_pass_df[['PACT_CLASS', 'PACT_MED', 'CR_TESTED']]
# Save X and y to file
X.to_csv(DIRPATH.format('X.csv'))
y.to_csv(DIRPATH.format('y.csv'))
# Output final dataset shape
print('X / Y SHAPE: {} / {}'.format(X.shape, y.shape))

# Split training and test sets
X_tr, X_t, y_tr, y_t = train_test_split(X, y, test_size=0.25, random_state=SEED)
# Save splittings to file
X_tr.to_csv(DIRPATH.format('X_train.csv'))
y_tr.to_csv(DIRPATH.format('y_train.csv'))
X_t.to_csv(DIRPATH.format('X_test.csv'))
y_t.to_csv(DIRPATH.format('y_test.csv'))
# Output dataset splitting sizes
print('TRAIN / TEST SET SIZES: {} / {}'.format(len(y_tr), len(y_t)))

In [ ]:
# Columnwise matrix quantile discretization (~= KBinsDiscretizer ordinal default behavior)
def disc_mat(mat, n_bins):
    bin_edges = np.quantile(mat, np.linspace(0, 1, n_bins + 1), axis=0)
    return np.stack([np.searchsorted(bin_edges[:, i], mat[:, i]) for i in range(mat.shape[1])], axis=1)

# Calculate a row of NMI scores for parallelization
def calc_nmi_row(mat, i):
    return (i, [metrics.normalized_mutual_info_score(mat[:, i], mat[:, j]) if j > i else 0 for j in range(mat.shape[1])])

# Parallelized pairwise normalized mutual info calculation with feature discretization, I/O as dataframes
def calc_nmi_df(df, n_bins=10):
    # Calculate NMI values for discretized features by row in parallel jobs
    rows = Parallel(n_jobs=N_JOBS)(delayed(calc_nmi_row)(disc_mat(df.values, n_bins), i) for i in range(df.shape[1]))
    # Sort and stack parallel job outputs
    nmi = np.stack([r[1] for r in sorted(rows, key=lambda r: r[0])])
    # Mirror over diagonal
    nmi = nmi + nmi.T
    # Add NMI of 1 for each feature with itself
    np.fill_diagonal(nmi, 1)
    # Return as dataframe
    return pd.DataFrame(nmi, columns=df.columns, index=df.columns)

# Plot heatmap of descriptor NMI to examine entanglements
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(data=calc_nmi_df(X_tr), square=True, ax=ax)
ax.set_title('Normalized Mutual Information of Descriptors')
ax.set_ylabel('')
plt.show()

In [ ]:
# Define algorithm, fixed parameters, and hyperparameter search grid for random forest model
rfc = RandomForestClassifier(criterion='entropy', oob_score=metrics.median_absolute_error, random_state=SEED)
rfc_grid = {
    'min_samples_leaf': [1, 2, 4, 8], 
    'n_estimators': [50, 100, 500], 
    'max_samples': [0.33, 0.5, 0.67, 1.0],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

# Do grid search CV over hyperparameters and calculate scores, optimizing for RMSE
scoring = ['neg_root_mean_squared_error', 'neg_mean_absolute_error', 'neg_median_absolute_error']
refit = 'neg_root_mean_squared_error'
rfc_search = GridSearchCV(estimator=rfc, param_grid=rfc_grid, scoring=scoring, refit=refit, n_jobs=N_JOBS).fit(X_tr, y_tr['PACT_CLASS'])

# Print CV results, params, and scores
rfc_search_results = pd.DataFrame(rfc_search.cv_results_).iloc[rfc_search.best_index_, :]
print('RFC OOB SCORE:\n{:g}'.format(rfc_search.best_estimator_.oob_score_))
print('RFC CV RESULTS:\n{}'.format(rfc_search_results))

In [ ]:
# Get test predictions
y_t['PACT_CLASS_PRED'] = rfc_search.predict(X_t)
y_t['PACT_CLASS_RES'] = y_t['PACT_CLASS_PRED'] - y_t['PACT_CLASS']
y_t['PACT_CLASS_ABS_ERR'] = abs(y_t['PACT_CLASS_RES'])
# Print external test scores
print('EXTERNAL TEST SCORE (RMSE) (CU): {:g}'.format(metrics.root_mean_squared_error(y_t['PACT_CLASS'], y_t['PACT_CLASS_PRED'])))
print('EXTERNAL TEST SCORE (MAE) (CU): {:g}'.format(metrics.mean_absolute_error(y_t['PACT_CLASS'], y_t['PACT_CLASS_PRED'])))
print('EXTERNAL TEST SCORE (MdAE) (CU): {:g}'.format(metrics.median_absolute_error(y_t['PACT_CLASS'], y_t['PACT_CLASS_PRED'])))

# Crosstabs for heatmap generation
ct_pred = pd.crosstab(y_t['PACT_CLASS_PRED'], y_t['PACT_CLASS'])
ct_pred[ct_pred == 0] = np.nan
ct_res = pd.crosstab(y_t['PACT_CLASS_RES'], y_t['PACT_CLASS'])
ct_res[ct_res == 0] = np.nan

# Plot heatmaps of predictions and residuals vs. true classes
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
sns.heatmap(data=ct_pred, annot=True, square=True, cmap='Blues', cbar=False, ax=ax[0])
sns.heatmap(data=ct_res, annot=True, square=True, cmap='Reds', cbar=False, ax=ax[1])
ax[0].set_xlabel('True Classes')
ax[1].set_xlabel('True Classes')
ax[0].set_ylabel('Class Predictions')
ax[1].set_ylabel('Class Residuals')
ax[0].invert_yaxis()
ax[1].invert_yaxis()
plt.show()

In [ ]:
# Identify adversarial examples in test set
# FP = true <= 20%, predicted 9-10
# FN = true >= 85%, predicted 0-2
y_t['ADV_TYPE'] = 'NA'
y_t['ADV_TYPE'] = y_t['ADV_TYPE'].case_when(caselist=[
    ((y_t['PACT_MED'] <= 20) & (y_t['PACT_CLASS_PRED'] >= 9), 'FP'),
    ((y_t['PACT_MED'] >= 85) & (y_t['PACT_CLASS_PRED'] <= 2), 'FN')
])

# Write adversarial example data to file and output for examination
y_t_adv = y_t[y_t['ADV_TYPE'] != 'NA'].sort_values('ADV_TYPE')
y_t_adv.to_csv(DIRPATH.format('adversarial_analysis.csv'))
y_t_adv.head(y_t_adv.shape[0])

In [ ]:
# Calculate proximity distance of points in X1 to points in X2 for a given random forest model
def calc_prox_dist(rf, X1, X2):
    # Hamming distance = fraction of indices where classes disagree
    return cdist(rf.apply(X1), rf.apply(X2), 'hamming')

# Generate training-test chemical proximity distance matrix
t_tr_prox_df = pd.DataFrame(calc_prox_dist(rfc_search.best_estimator_, X_t, X_tr), index=X_t.index, columns=X_tr.index)
# Identify the most proximal training chemical for each test chemical
t_tr_prox_df['MIN_PROX_DIST'] = np.min(t_tr_prox_df, axis=1)
t_tr_prox_df['NEAREST_NEIGHBOR'] = t_tr_prox_df.columns[np.argmin(t_tr_prox_df, axis=1)]

# Output minimum proximity distance and nearest neighbor for all adversarial examples previously identified
t_tr_adv_prox_df = t_tr_prox_df.loc[y_t_adv.index, ['MIN_PROX_DIST', 'NEAREST_NEIGHBOR']]
t_tr_adv_prox_df.head(t_tr_adv_prox_df.shape[0])

In [ ]:
# Theoretical maximum entropy of a uniform distribution across bins
s_max_theory = entropy([1 / (n_bins + 2)] * (n_bins + 2), base=2)
# Maximum entropy based on the prior distribution of the training set
# s_max_prior = entropy(np.unique(y_tr['PACT_CLASS'], return_counts=True)[1] / y_tr.shape[0], base=2) 

# Compute entropy of test set prediction probability distributions
y_t['PRED_S_NORM'] = entropy(rfc_search.predict_proba(X_t), axis=1, base=2) / s_max_theory

# Compute MAE, MdAE, and fraction covered over entropy thresholds
y_t_ad = y_t.sort_values('PRED_S_NORM')
y_t_ad['PACT_CLASS_MAE'] = y_t_ad['PACT_CLASS_ABS_ERR'].expanding().mean()
y_t_ad['PACT_CLASS_MDAE'] = y_t_ad['PACT_CLASS_ABS_ERR'].expanding().median()
y_t_ad['FRAC_COVER'] = y_t_ad['PACT_CLASS_ABS_ERR'].expanding().count() / y_t_ad.shape[0]

# Plot error and coverage against entropy threshold
fig, ax = plt.subplots()
sns.lineplot(data=y_t_ad, x='PRED_S_NORM', y='PACT_CLASS_MAE', ax=ax, label='MAE')
sns.lineplot(data=y_t_ad, x='PRED_S_NORM', y='PACT_CLASS_MDAE', ax=ax, label='MdAE')
ax.set_xlabel('Normalized Prediction Entropy (nPS) Threshold')
ax.set_ylabel('Prediction Error (CU)')
ax_r = plt.twinx()
sns.lineplot(data=y_t_ad, x='PRED_S_NORM', y='FRAC_COVER', color='black', linestyle='--', ax=ax_r, label='Test Set Coverage')
ax_r.set_ylabel('Fraction of Test Set')
ax_r.legend(loc='lower right')
plt.show()

# Output 10 highest-entropy predictions for examination
y_t_ad.tail(10)

In [ ]:
# Binarize predictions for prioritization and generate confusion matrix
y_t['CR_RECOMMENDED'] = np.where(y_t['PACT_CLASS_PRED'] >= 8, 'Y', 'N')
metrics.ConfusionMatrixDisplay.from_predictions(y_t['CR_TESTED'], y_t['CR_RECOMMENDED'], labels=['Y', 'N'])

# Examine predictions for compounds with experimental activity greater than T4
gtt4_df = pd.read_csv(DIRPATH.format('greater_than_t4.csv')).set_index('DTXSID')
y_t[y_t.index.isin(gtt4_df.index)].head(gtt4_df.shape[0])

In [ ]:
# Calculate permutation importance and mutual information of features over the test set
pi = permutation_importance(rfc_search.best_estimator_, X_t, y_t['PACT_CLASS'], n_repeats=100, random_state=SEED, n_jobs=N_JOBS)
impt_df = pd.DataFrame({
    'FEATURE': embedding,
    'IMPORTANCE': pi['importances_mean'],
    'SD': pi['importances_std'],
    'SIGNIFICANCE': pi['importances_mean'] - pi['importances_std'],
    'MIC': mutual_info_classif(X_t, y_t['PACT_CLASS'], random_state=SEED)
}).set_index('FEATURE').sort_values('MIC', ascending=False)
# Output top 10
impt_df.head(10)

In [ ]:
pfas_df = pd.read_csv(DIRPATH.format('PFAS8a7v3.csv')).set_index('DTXSID')
pfas_data_df = sc_df[['QualityCall', 'CONC_MAX', 'PACT_MED', 'CR_TESTED']][sc_df.index.isin(pfas_df.index)]
pfas_data_df['PACT_CLASS'] =  np.digitize(pfas_data_df['PACT_MED'], bin_edges)
pd_all_df = pd.concat([pd_df, pd.read_csv(DIRPATH.format('bad_qsar_QSAR-ready_smi_PadelDesc.csv')).set_index('Name')])
pfas_data_df = pfas_data_df.join(pd_all_df[~pd_all_df.index.duplicated(keep='first')], how='inner')
pfas_data_df = pfas_data_df.loc[~pfas_data_df.index.isin(X_tr.index) & (pfas_data_df['QualityCall'] != 'Fail'), :]

X_pfas = pfas_data_df[embedding]
y_pfas = pfas_data_df[['PACT_CLASS', 'PACT_MED']]

y_pfas['PACT_CLASS_PRED'] = rfc_search.predict(X_pfas)
y_pfas['PACT_CLASS_RES'] = y_pfas['PACT_CLASS_PRED'] - y_pfas['PACT_CLASS']
y_pfas['PACT_CLASS_ABS_ERR'] = abs(y_pfas['PACT_CLASS_RES'])
y_pfas['PRED_S_NORM'] = entropy(rfc_search.predict_proba(X_pfas), axis=1, base=2) / s_max_theory
y_pfas.head(y_pfas.shape[0])

In [ ]:
# Read ToxCast Phase III list from file and do prediction and prioritization
ph3_df = pd.read_csv(DIRPATH.format('ph3-list_PadelDesc.csv')).set_index('Name')[embedding]
ph3_df['PACT_CLASS_PRED'] = rfc_search.predict(ph3_df)

# Calculate prioritization, probability, entropy, and nondominated rank for Phase III predictions
ph3_prob = rfc_search.predict_proba(ph3_df[embedding])
ph3_df['PRED_P_INACT'] = np.sum(ph3_prob[:, :3], axis=1)
ph3_df['PRED_P_MOD'] = np.sum(ph3_prob[:, 3:8], axis=1)
ph3_df['PRED_P_ACT'] = np.sum(ph3_prob[:, -3:], axis=1)
ph3_df['PRED_S_NORM'] = entropy(ph3_prob, axis=1, base=2) / s_max_theory
ph3_df['ND_RANK_INACT_HS'] = nondominated_rank(ph3_df[['PRED_P_INACT', 'PRED_S_NORM']].values, larger_is_better_objectives=[0, 1])
ph3_df['ND_RANK_MOD_HS'] = nondominated_rank(ph3_df[['PRED_P_MOD', 'PRED_S_NORM']].values, larger_is_better_objectives=[0, 1])
ph3_df['ND_RANK_ACT_HS'] = nondominated_rank(ph3_df[['PRED_P_ACT', 'PRED_S_NORM']].values, larger_is_better_objectives=[0, 1])
ph3_df['ND_RANK_INACT_LS'] = nondominated_rank(ph3_df[['PRED_P_INACT', 'PRED_S_NORM']].values, larger_is_better_objectives=[0])
ph3_df['ND_RANK_MOD_LS'] = nondominated_rank(ph3_df[['PRED_P_MOD', 'PRED_S_NORM']].values, larger_is_better_objectives=[0])
ph3_df['ND_RANK_ACT_LS'] = nondominated_rank(ph3_df[['PRED_P_ACT', 'PRED_S_NORM']].values, larger_is_better_objectives=[0])
# Save to file
ph3_df.to_csv(DIRPATH.format('ph3-list_predictions.csv'))

fig, ax = plt.subplots(1, 3, figsize=(12, 4), sharex=True)
sns.scatterplot(data=ph3_df, y='PRED_P_INACT', x='PRED_S_NORM', hue='ND_RANK_INACT_HS', palette='viridis_r', ax=ax[0])
sns.scatterplot(data=ph3_df, y='PRED_P_MOD', x='PRED_S_NORM', hue='ND_RANK_MOD_HS', palette='viridis_r', ax=ax[1])
sns.scatterplot(data=ph3_df, y='PRED_P_ACT', x='PRED_S_NORM', hue='ND_RANK_ACT_HS', palette='viridis_r', ax=ax[2])
ax[0].set_ylabel('$P_{0:2}$')
ax[1].set_ylabel('$P_{3:7}$')
ax[2].set_ylabel('$P_{8:10}$')
for a in ax:
    a.set_xlabel('$nPS$')
    a.legend(title='ND Rank')
plt.suptitle('Nondominated Rank of Predicted Inactive, Moderate, and Active Chemicals for Model Improvement')
plt.show()

In [ ]:
# Define algorithm, fixed parameters, and hyperparameter search grid for random forest regressor model
rfr = RandomForestRegressor(oob_score=metrics.median_absolute_error, random_state=SEED)
rfr_grid = {
    'min_samples_leaf': [1, 2, 4, 8], 
    'n_estimators': [50, 100, 500], 
    'max_samples': [0.33, 0.5, 0.67, 1.0]
}

# Do grid search CV over hyperparameters and calculate scores, optimizing for RMSE
rfr_search = GridSearchCV(estimator=rfr, param_grid=rfr_grid, scoring=scoring, refit=refit, n_jobs=N_JOBS).fit(X_tr, y_tr['PACT_MED'])

# Print CV results, params, and scores
rfr_search_results = pd.DataFrame(rfr_search.cv_results_).iloc[rfr_search.best_index_, :]
print('RFR OOB SCORE:\n{:g}'.format(rfr_search.best_estimator_.oob_score_))
print('RFR CV RESULTS:\n{}'.format(rfr_search_results))

In [ ]:
# Get test predictions from regressor
y_t['PACT_MED_PRED'] = rfr_search.predict(X_t)
# Print external test scores
print('EXTERNAL TEST SCORE (RMSE) (AU): {:g}'.format(metrics.root_mean_squared_error(y_t['PACT_MED'], y_t['PACT_MED_PRED'])))
print('EXTERNAL TEST SCORE (MAE) (AU): {:g}'.format(metrics.mean_absolute_error(y_t['PACT_MED'], y_t['PACT_MED_PRED'])))
print('EXTERNAL TEST SCORE (MdAE) (AU): {:g}'.format(metrics.median_absolute_error(y_t['PACT_MED'], y_t['PACT_MED_PRED'])))

# Discretize predictions for comparison
y_t['PACT_MED_PRED_CLASS'] = np.digitize(y_t['PACT_MED_PRED'], bin_edges)
# Print external test scores
print('EXTERNAL TEST SCORE (RMSE) (CU): {:g}'.format(metrics.root_mean_squared_error(y_t['PACT_CLASS'], y_t['PACT_MED_PRED_CLASS'])))
print('EXTERNAL TEST SCORE (MAE) (CU): {:g}'.format(metrics.mean_absolute_error(y_t['PACT_CLASS'], y_t['PACT_MED_PRED_CLASS'])))
print('EXTERNAL TEST SCORE (MdAE) (CU): {:g}'.format(metrics.median_absolute_error(y_t['PACT_CLASS'], y_t['PACT_MED_PRED_CLASS'])))

# Generate true, classifier, and regressor predicted activity distribution plots
fig, ax = plt.subplots(1, 3, figsize=(12, 4), sharex=True, sharey=True)
sns.histplot(data=y_t, x='PACT_MED', bins=plt_bin_edges, ax=ax[0], kde=True)
y_t['PACT_CLASS_PRED_VAL'] = cls_agg_df['PACT_MED']['median'].loc[y_t['PACT_CLASS_PRED']].reset_index().iloc[:, 1].values
sns.histplot(data=y_t, x='PACT_CLASS_PRED_VAL', bins=plt_bin_edges, ax=ax[1], kde=True)
sns.histplot(data=y_t, x='PACT_MED_PRED', bins=plt_bin_edges, ax=ax[2], kde=True)
ax[0].set_title('True Activity Distribution')
ax[1].set_title('Classifier-Predicted Activity Distribution')
ax[2].set_title('Regressor-Predicted Activity Distribution')
# Add bin labels and x axis labels for all subplots
for a in ax:
    a.set_xlabel('Median % Activity')
    for i in range(len(plt_bin_edges) - 1):
        a.text((plt_bin_edges[i] + plt_bin_edges[i + 1]) / 2, 3, str(int(i)), ha='center', va='bottom')
plt.show()

In [ ]:
# Binarize regressor predictions for comparison
y_t['CR_RECOMMENDED_REG'] = np.where(y_t['PACT_MED_PRED'] >= 85, 'Y', 'N')
# y_t['CR_RECOMMENDED_REG'] = np.where(y_t['PACT_MED_PRED'] >= 69, 'Y', 'N')
# y_t['CR_RECOMMENDED_REG'] = np.where(y_t['PACT_MED_PRED_CLASS'] >= 8, 'Y', 'N')
# Generate confusion matrix
metrics.ConfusionMatrixDisplay.from_predictions(y_t['CR_TESTED'], y_t['CR_RECOMMENDED_REG'], labels=['Y', 'N'])

# Examine predictions for compounds with experimental activity greater than T4
y_t[y_t.index.isin(gtt4_df.index)].head(gtt4_df.shape[0])

In [ ]:
# Get test predictions from basic kNN classifier
knc = KNeighborsClassifier(algorithm='brute').fit(X_tr, y_tr['PACT_CLASS'])
y_t['PACT_CLASS_PRED_KNN'] = knc.predict(X_t)

# Get test predictions from basic kNN regressor
knr = KNeighborsRegressor(algorithm='brute').fit(X_tr, y_tr['PACT_MED'])
y_t['PACT_MED_PRED_KNN'] = knr.predict(X_t)

# Test if same neighbors were selected by classifier and regressor
print('NEIGHBOR ARRAYS EQUAL: {}'.format(np.array_equal(knc.kneighbors(X_t)[1], knr.kneighbors(X_t)[1])))

# Print external test scores
print('KNC EXTERNAL TEST SCORE (RMSE) (CU): {:g}'.format(metrics.root_mean_squared_error(y_t['PACT_CLASS'], y_t['PACT_CLASS_PRED_KNN'])))
print('KNC EXTERNAL TEST SCORE (MAE) (CU): {:g}'.format(metrics.mean_absolute_error(y_t['PACT_CLASS'], y_t['PACT_CLASS_PRED_KNN'])))
print('KNC EXTERNAL TEST SCORE (MdAE) (CU): {:g}'.format(metrics.median_absolute_error(y_t['PACT_CLASS'], y_t['PACT_CLASS_PRED_KNN'])))

print('KNR EXTERNAL TEST SCORE (RMSE) (AU): {:g}'.format(metrics.root_mean_squared_error(y_t['PACT_MED'], y_t['PACT_MED_PRED_KNN'])))
print('KNR EXTERNAL TEST SCORE (MAE) (AU): {:g}'.format(metrics.mean_absolute_error(y_t['PACT_MED'], y_t['PACT_MED_PRED_KNN'])))
print('KNR EXTERNAL TEST SCORE (MdAE) (AU): {:g}'.format(metrics.median_absolute_error(y_t['PACT_MED'], y_t['PACT_MED_PRED_KNN'])))

# Discretize predictions for comparison
y_t['PACT_MED_PRED_KNN_CLASS'] = np.digitize(y_t['PACT_MED_PRED_KNN'], bin_edges)
# Print external test scores
print('KNR EXTERNAL TEST SCORE (RMSE) (CU): {:g}'.format(metrics.root_mean_squared_error(y_t['PACT_CLASS'], y_t['PACT_MED_PRED_KNN_CLASS'])))
print('KNR EXTERNAL TEST SCORE (MAE) (CU): {:g}'.format(metrics.mean_absolute_error(y_t['PACT_CLASS'], y_t['PACT_MED_PRED_KNN_CLASS'])))
print('KNR EXTERNAL TEST SCORE (MdAE) (CU): {:g}'.format(metrics.median_absolute_error(y_t['PACT_CLASS'], y_t['PACT_MED_PRED_KNN_CLASS'])))

# Generate true, classifier, and regressor predicted activity distribution plots
fig, ax = plt.subplots(1, 3, figsize=(12, 4), sharex=True, sharey=True)
sns.histplot(data=y_t, x='PACT_MED', bins=plt_bin_edges, ax=ax[0], kde=True)
y_t['PACT_CLASS_PRED_KNN_VAL'] = cls_agg_df['PACT_MED']['median'].loc[y_t['PACT_CLASS_PRED_KNN']].reset_index().iloc[:, 1].values
sns.histplot(data=y_t, x='PACT_CLASS_PRED_KNN_VAL', bins=plt_bin_edges, ax=ax[1], kde=True)
sns.histplot(data=y_t, x='PACT_MED_PRED_KNN', bins=plt_bin_edges, ax=ax[2], kde=True)
ax[0].set_title('True Activity Distribution')
ax[1].set_title('kNN Classifier-Predicted Activity Distribution')
ax[2].set_title('kNN Regressor-Predicted Activity Distribution')
# Add bin labels and x axis labels for all subplots
for a in ax:
    a.set_xlabel('Median % Activity')
    for i in range(len(plt_bin_edges) - 1):
        a.text((plt_bin_edges[i] + plt_bin_edges[i + 1]) / 2, 3, str(int(i)), ha='center', va='bottom')
plt.show()

In [ ]:
# Reduce features using importance results on test set
impt_embedding = impt_df[impt_df['MIC'] > 0.1].index
# impt_embedding = impt_df[impt_df['SIGNIFICANCE'] > 0.001].index

# Define UMAP pipeline with feature scaling
umap_pipe = Pipeline([('scaler', StandardScaler()), ('umap', umap.UMAP(random_state=SEED))])
# Fit UMAP pipeline to training set
y_tr[['UMAP1', 'UMAP2']] = umap_pipe.fit_transform(X_tr[impt_embedding])
# Apply fit UMAP pipeline to test set
y_t[['UMAP1', 'UMAP2']] = umap_pipe.transform(X_t[impt_embedding])
# UMAP transform on Phase III compounds
ph3_df[['UMAP1', 'UMAP2']] = umap_pipe.transform(ph3_df[impt_embedding])

# Generate UMAP plot of Phase III compounds alongside training and test sets
fig, ax = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)
mk = {'NA': 'o', 'FP': '^', 'FN': 'X'}
sns.scatterplot(data=y_tr, x='UMAP1', y='UMAP2', hue='PACT_CLASS', palette='RdBu_r', ax=ax[0])
sns.scatterplot(data=y_t, x='UMAP1', y='UMAP2', hue='PACT_CLASS', style='ADV_TYPE', palette='RdBu_r', markers=mk, ax=ax[1])
sns.scatterplot(data=ph3_df, x='UMAP1', y='UMAP2', markers=mk, ax=ax[2])
plt.show()